In [2]:
from phonemizer import phonemize
import Levenshtein
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
import torch
import librosa
import numpy as np

# MFA 포맷을 IPA로 변환하기 위한 딕셔너리
MFA2IPA = {
    'A': 'ɐ',
    'iA': 'jɐ',
    'oA': 'wɐ',
    'E': 'ɛ',
    'iE': 'je',
    'oE': 'wɛ',
    'I': 'i',
    'uI': 'wi',
    'O': 'o',
    'iO': 'jo',
    'U': 'u',
    'iU': 'ju',
    'EO': 'ʌ',
    'iEO': 'jʌ',
    'uEO': 'wʌ',
    'EU': 'ɯ',
    'euI': 'ɯj',
    'G': 'q',
    'N': 'n',
    'D': 'd',
    'R': 'ɾ',
    'M': 'm',
    'B': 'p',
    'S': 's',
    'J': 'tɕ',
    'Kh': 'kh',
    'Th': 'th',
    'Ph': 'ph',
    'H': 'h',
    'GG': 'qo',
    'DD': 't',
    'CHh': 'tʃh',
    'NG': 'ŋ',
    'L':'ɫ',
    'p':'p',
}


# 한글 텍스트를 IPA로 변환
def convert_to_ipa(korean_text):
    ipa_text = phonemize(korean_text, language="ko", backend="espeak")

    return ipa_text

# Wav2Vec2로 오디오 파일에서 MFA 추론
def map_to_pred(audio):
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to('cuda')

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription 

# 발음 기호간 유사도 계산
def calculate_similarity(keyword_ipa, predicted_ipa):
    distance = 0
    keyword_length = len(keyword_ipa)
    prediction_length = len(predicted_ipa)
    
    # Levenshtein 거리를 계산할 때 윈도우를 옮겨가며 최대가 되는 지점을 사용
    # 예측된 것이 키워드의 IPA보다 작거나 같으면 윈도우 없이 그냥 계산
    if keyword_length >= prediction_length:
        # 문자열 길이 15로 패딩
        padded_keyword_ipa = keyword_ipa+'0'*(15-keyword_length) if keyword_length < 15 else keyword_ipa
        padded_predicted_ipa = predicted_ipa+'0'*(15-prediction_length) if prediction_length < 15 else predicted_ipa
        
        # Levenshtein 거리 계산
        distance = Levenshtein.ratio(padded_keyword_ipa, padded_predicted_ipa)
    else:
        for window in range(len(predicted_ipa) - keyword_length):
            # 문자열 길이 15로 패딩
            padded_keyword_ipa = keyword_ipa+'0'*(15-keyword_length) if keyword_length < 15 else keyword_ipa
            padded_predicted_ipa = predicted_ipa[window:window+keyword_length+1]+'0'*(15-keyword_length+1) if keyword_length+1 < 15 else predicted_ipa[window:window+keyword_length+1]
            
            # Levenshtein 거리 계산
            distance = max(distance, Levenshtein.ratio(padded_keyword_ipa, padded_predicted_ipa))
    
    return distance


# 문자열에서 IPA 받아옴 ex) 영석아 -> jʌŋsʌqɐ
keyword = "저기요"
ipa_of_keyword = convert_to_ipa(keyword).replace(' ', '')
print(f"ipa of '{keyword}' : {ipa_of_keyword}")

# 허깅페이스에서 Wav2Vec2로 한국어의 MFA 발음을 예측하도록 학습된 모델을 가져옴
processor = Wav2Vec2Processor.from_pretrained("slplab/wav2vec2-xls-r-300m_phone-mfa_korean")
model = Wav2Vec2ForCTC.from_pretrained("slplab/wav2vec2-xls-r-300m_phone-mfa_korean").to('cuda')

/home/20223164/.conda/envs/ssl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: espeak not installed on your system

In [ ]:
# 오디오 로드
audio, sr = librosa.load('./영석아3.wav', sr=16000)

# 모델로 입력 오디오의 MFA format 추론
result = map_to_pred([audio])

# MFA format을 IPA symbol로 변환
model_output_ipa = ''.join([MFA2IPA[mfa] for mfa in result[0].split(' ')])
print(f"ipa of model prediction : {model_output_ipa}")

# Levenshtein 거리로 유사도 계산
# 임계값은 0.9로 설정
similarity = calculate_similarity(ipa_of_keyword, model_output_ipa)
print(f"similarity between '{ipa_of_keyword}' and '{model_output_ipa}' : {similarity}")